# Partie 3 : Analyse de Données Séquentielles avec le Deep Learning (NLP)

## 3.1. Contexte et Objectifs
Ce dernier module aborde les données séquentielles via le Traitement Automatique du Langage (NLP). L'ordre des mots étant crucial, nous utiliserons des architectures spécifiques comme les Réseaux de Neurones Récurrents (RNN) et leur variante avancée, les Long Short-Term Memory (LSTM), qui sont capables de mémoriser des informations sur de longues séquences.

Votre objectif sera de :
- Maîtriser le pipeline de prétraitement de texte avec `TextVectorization`.
- Comprendre le rôle de la couche `Embedding` pour la représentation sémantique des mots.
- Construire et entraîner un modèle de classification de texte basé sur une architecture `LSTM`.

## 3.2. Cas d'Usage : Catégorisation Automatique de Dépenses
Pour ce projet, pourriez-vous utiliser un jeu de données de catégorisation de dépenses comme celui-ci sur Kaggle ? Il contient des descriptions textuelles de transactions (ex: "STARBUCKS COFFEE") et leur catégorie (ex: "Food") . Votre but est de prédire la catégorie à partir de la description.

## 3.3. Implémentation Guidée

### Étape 1 : Prétraitement du Texte
L'approche moderne consiste à intégrer le prétraitement dans le modèle. Pourriez-vous utiliser la couche `tf.keras.layers.TextVectorization` pour standardiser, tokeniser et vectoriser le texte ?
```python
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Charger les données (exemple avec le dataset de Kaggle)
df = pd.read_csv('personal_expense_dataset.csv')


df = df.dropna()

# Encoder les labels textuels en entiers
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['Category'])
num_classes = len(label_encoder.classes_)

# Séparer les données
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df,
    df['category_encoded'],
    test_size=0.2,
    random_state=42
)

# Créer et adapter la couche de vectorisation
max_features = 10000
sequence_length = 50

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

# Adapter la couche au vocabulaire des textes d'entraînement
vectorize_layer.adapt(train_texts.values)
```

### Étape 2 : Construction du Modèle Séquentiel
Votre tâche est maintenant de construire le modèle. Pourriez-vous utiliser la structure suivante ?
1. La couche `TextVectorization` que vous venez de créer.
2. Une couche `Embedding` pour apprendre des représentations vectorielles denses des mots.
3. Une couche `Bidirectional(LSTM)` pour analyser la séquence dans les deux sens, capturant ainsi un contexte plus riche.
4. Des couches `Dense` pour la classification finale.

```python
embedding_dim = 128
model = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    layers.Embedding(input_dim=max_features + 1, output_dim=embedding_dim),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax') # Softmax pour la classification multi-classe
])
```

### Étape 3 : Callbacks, Compilation et Entraînement
Comme pour la partie 1, pourriez-vous configurer des callbacks `ModelCheckpoint` et `EarlyStopping` ? Ensuite, compilez et entraînez le modèle.

```python
# Callbacks
nlp_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_nlp.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

nlp_early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Compilation
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# Entraînement
epochs = 10
history = model.fit(
    train_texts,
    train_labels,
    validation_data=(test_texts, test_labels),
    epochs=epochs,
    callbacks=[nlp_checkpoint_callback, nlp_early_stopping_callback]
)
```